# ⛄Frosty Friday Week28 🍂 

## 使うデータソース

COVID-19の疫学情報は以下を用いる。  
[COVID-19 Epidemiological Data](https://app.snowflake.com/marketplace/listing/GZSNZ7F5UH/starschema-covid-19-epidemiological-data)
データベース名は ```FFW28_COVID19``` に変更しておく。  

気象情報は以下を用いる。  
[Weather & Environment](https://app.snowflake.com/marketplace/listing/GZTSZAS2KIM/cybersyn-weather-environment)
データベース名は ```FFW28_WEATHER``` に変更しておく。  

## ステップ1 "とりあえずわかりやすいところから"

まず、ISO 3166-1 alpha-2 codeに変換したい国名を列挙する。  

In [ ]:
SELECT DISTINCT(COUNTRY_NAME) AS COUNTRY_NAME
FROM FFW28_WEATHER.CYBERSYN.NOAA_WEATHER_STATION_INDEX;

## ステップ2 "睡眠こそ最強の解決策である。それでダメならLLMがなんとかしてくれる"

Snowflake Cortex LLM Functionsで国名をISO 3166-1 alpha-2 codeに変換する。  
色々試してみるといいけど、この手のシンプルタスクは結局 Snowflake Arctic が一番安定するんだよなぁ。  
ただし、出力の1文字目に半角スペースが追加されがちなので注意！  

In [ ]:
SELECT COUNTRY_NAME, SUBSTRING(SNOWFLAKE.CORTEX.COMPLETE('snowflake-arctic', CONCAT('You are an assistant that converts country names to ISO 3166-1 alpha-2 codes. Follow these rules:

1. Output ONLY the ISO 3166-1 alpha-2 code (2 characters) for the input country name
2. Do not output any explanatory text or additional information
3. Output only 2 uppercase alphabetic characters
4. If the code is unknown, output "XX"

Sample format:
### Sample Input ###
Japan
### Sample Output ###
JP

### Sample Input ###
United States of America
### Sample Output ###
US

### Actual Input ###
', COUNTRY_NAME)), 2, 2) AS ISO3166_1
FROM {{cell1}};


## ステップ3 "決まってるだろ…合体だ！"

まずは二つのデータソースの国名をJOINする。

In [ ]:
SELECT
    DISTINCT(A.COUNTRY_REGION) AS COUNTRY_NAME_1,
    B.COUNTRY_NAME AS COUNTRY_NAME_2,
    A.ISO3166_1
FROM FFW28_COVID19.PUBLIC.ECDC_GLOBAL AS A
JOIN {{cell2}} AS B
ON A.ISO3166_1 = B.ISO3166_1;

## ステップ4 "困難は分割せよ"

次に観測局の方を整形していこう。  
国ごとに "highest" なものを抽出する。

In [ ]:
SELECT * FROM
(
SELECT
    NOAA_WEATHER_STATION_ID AS STATION_ID,
    NOAA_WEATHER_STATION_NAME AS STATION_NAME,
    COUNTRY_NAME,
    LATITUDE,
    LONGITUDE,
    RANK() OVER (PARTITION BY COUNTRY_NAME ORDER BY NOAA_WEATHER_STATION_NAME DESC) AS RANK_NUM
FROM FFW28_WEATHER.CYBERSYN.NOAA_WEATHER_STATION_INDEX
)
WHERE RANK_NUM = 1;

## ステップ5 "RANK IS 万能"

それぞれの観測局の最大の平均気温を抽出する。  
VALUEで ```PARTITION BY``` して最新の行を取得する。

In [ ]:
SELECT * FROM
(
SELECT
    NOAA_WEATHER_STATION_ID AS STATION_ID,
    DATE,
    VALUE,
    RANK() OVER (PARTITION BY STATION_ID ORDER BY VALUE DESC) AS RANK_NUM
FROM FFW28_WEATHER.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES
WHERE VARIABLE_NAME LIKE '%Average Temperature%'
)
WHERE RANK_NUM = 1;

## ステップ6 "全部合体！バーン！バーン！さらにバーン！"

準備ができたので全部JOINしていく。

In [ ]:
WITH STATION_AND_TEMPERATURE AS
(
    SELECT
        A.COUNTRY_NAME,
        A.STATION_ID,
        A.STATION_NAME,
        A.LATITUDE,
        A.LONGITUDE,
        B.DATE,
        B.VALUE
    FROM {{cell4}} A
    JOIN {{cell5}} B
    ON A.STATION_ID = B.STATION_ID
)
, RESULT_TABLE AS
(
    SELECT
        C.COUNTRY_NAME_1 AS COUNTRY_NAME,
        C.ISO3166_1,
        D.STATION_ID,
        D.STATION_NAME,
        D.LATITUDE,
        D.LONGITUDE,
        D.DATE,
        D.VALUE
    FROM {{cell3}} C
    JOIN STATION_AND_TEMPERATURE D
    ON C.COUNTRY_NAME_2 = D.COUNTRY_NAME
)
SELECT * FROM RESULT_TABLE;

## ステップ7 "ピーナッツはすき。ヤード・ポンド法はきらい。"

国名でソートしよう。  
摂氏(℃)じゃなくて華氏(℉)で出力することにも注意！  
参考：[℉] = [℃] × 9⁄5 + 32

In [ ]:
SELECT
    COUNTRY_NAME,
    ISO3166_1
    STATION_ID,
    STATION_NAME,
    LATITUDE,
    LONGITUDE,
    DATE,
    ((VALUE * 1.8) + 32) AS FAHRENHEIT_TEMPERATURE,
    ROW_NUMBER() OVER (PARTITION BY ISO3166_1 ORDER BY DATE DESC) AS ROW_NUM
FROM {{cell6}}
QUALIFY ROW_NUM = 1
ORDER BY COUNTRY_NAME;

## おまけ "ホコリかぶってた気温のリストもプロットしたら伝説じゃない"

Snowflake Notebooksなら可視化も簡単。  

In [ ]:
import streamlit as st

# Pandasデータフレームで前のセルの値を取得する
df = cell7.to_pandas()
st.dataframe(df)

# プロットで使う追加のカラムを作成
# 大きい値がより大きく見えるようにN乗にして強調する
df["size"] = (df["FAHRENHEIT_TEMPERATURE"] ** 2.5) * 5

# 地図にプロットする
st.map(df,
       latitude="LATITUDE",
       longitude="LONGITUDE",
       size="size")